In [22]:
import pandas as pd
import os
import glob
import numpy as np
from pathlib import Path
pd.set_option('display.max_columns', None)


In [2]:
#os.chdir('../')

In [68]:
def GetContributorData(indir, major_months, window):
    df_contributors = pd.read_parquet(indir / f'major_contributors_major_months{major_months}_window{window}D_samplefull.parquet')
    return df_contributors
    
def ContributorCount(df):
    return df[['actor_id','repo_name']].drop_duplicates().shape[0]

def GetConsecutiveSum(df):
    gb = df.groupby((df['issue_exceed'] != df['criteria_exceed'].shift()).cumsum())
    df['consecutive_periods'] = gb['criteria_exceed'].cumsum()
    df.loc[df['criteria_exceed'] == 0, 'consecutive_periods'] = 0
    return df

def GetCandidates(departure_candidates, post_period_length, decline_type, decline_threshold, decline_pct, criteria_analysis_col):
    df_candidates = pd.DataFrame()
    for i in departure_candidates.index:
        final_period = departure_candidates.loc[i, 'final_period']
        total_consecutive_periods = departure_candidates.loc[i, 'consecutive_periods']
        repo_name = departure_candidates.loc[i, 'repo_name']
        actor_id = departure_candidates.loc[i, 'actor_id']
        df_potential_consecutive_subset = df_potential_consecutive.query(f'repo_name == "{repo_name}" & actor_id == "{actor_id}"')
        df_candidate = pd.merge(df_potential_consecutive_subset, departure_candidates.loc[[i]].drop('consecutive_periods', axis = 1))

        final_periods = df_candidate.query('time_period>final_period').head(post_period_length)
        prior_periods = df_candidate.query('time_period<=final_period').sort_values('time_period').tail(total_consecutive_periods)
        if final_periods.shape[0]>post_period_length: 
            if final_periods[criteria_analysis_col].isna().sum() != 0: # project is still active
                if decline_type == "threshold_mean":
                    pre_period_mean = prior_periods[critera_col].mean()
                    post_period_mean = final_periods[critera_col].mean()
                    if pre_period_mean * decline_threshold > post_period_mean:
                        df_candidates = pd.concat([df_candidates, df_candidate])
                if decline_type == "threshold_pct":
                    decline_analysis_col = f'{criteria_col}_{decline_pct}th_pct'
                    final_periods_fulfill = final_periods.query(f'{criteria_col}<{decline_analysis_col}')
                    if final_periods_fulfill.shape[0] == post_period_length:
                        df_candidates = pd.concat([df_candidates, df_candidate])
    return df_candidates

In [69]:
indir = Path('drive/output/derived/major_contributor_prospects')
indir_truck = Path('drive/output/scrape/get_weekly_truck_factor')

major_months_list = [2, 3, 6, 12]
window_list = [367, 732, 1828, 3654]
consecutive_periods_major_months_dict = {2: [6, 9],
                                         3: [4, 8],
                                         6: [3, 6],
                                         12: [2, 3, 4]}
issue_col = 'issue_comments'
pr_col = 'pr'
criteria_col_list = [issue_col, pr_col]
criteria_pct_list = [75, 90]
general_pct_list = [25, 50]

post_period_length_list = [2, 4]
decline_threshold_list = [.1, .2, .3]
decline_pct_list = [.1, .25]

In [ ]:
df_truckfactor = pd.concat([pd.read_csv(file) for file in indir_truck.glob('*.csv')]).drop('Unnamed: 0', axis = 1)

In [ ]:
for major_months in major_months_list:
    for window in window_list:
        df_contributors = GetContributorData(indir, major_months, window)
        num_contributors = ContributorCount(df_contributors)
        for criteria_col in criteria_col_list:
            for criteria_pct in criteria_pct_list:
                for general_pct in general_pct_list:
                    criteria_analysis_col = f'{criteria_col}_{criteria_pct}th_pct'
                    criteria_general_col = f'general_{criteria_col}_{general_pct}th_pct'
                    criteria_cols = [criteria_col, criteria_analysis_col, criteria_general_col]

                    analysis_cols = identifiers + criteria_cols
                    df_analysis = df_contributors[analysis_cols]

                    potential_major_contributors = df_analysis.query(f'{criteria_col}>{criteria_analysis_col} & {criteria_col}>{criteria_general_col}')\
                        [['actor_id','repo_name']].drop_duplicates()
                    df_potential = pd.merge(df_analysis, potential_major_contributors, on = ['actor_id','repo_name'])
                    num_potential_contributors = ContributorCount(df_potential)
                    df_potential = df_potential.assign(criteria_exceed = (df_potential[criteria_col]>df_potential[criteria_analysis_col]).astype(int))
                    df_potential = df_potential.groupby(['actor_id','repo_name']).apply(GetConsecutiveSum).reset_index(drop = True)

                    consecutive_periods_list = consecutive_periods_major_months_dict[windpw]
                    for consecutive_periods in consecutive_periods_list:
                        has_consecutive_periods = df_potential.query(f'consecutive_periods>={consecutive_periods}')[['actor_id','repo_name']].drop_duplicates()
                        df_potential_consecutive = pd.merge(df_potential, has_consecutive_periods)
                        num_consecutive_periods = ContributorCount(df_potential_consecutive)

                        departure_candidates = df_potential_consecutive[['actor_id','repo_name', 'consecutive_periods', 'time_period']]\
                            .sort_values('consecutive_periods', ascending = False)\
                            .drop_duplicates(['actor_id','repo_name'])\
                            .reset_index(drop = True)\
                            .rename({'time_period':'final_period'}, axis = 1)
                        num_departure_candidates = ContributorCount(departure_candidates)
            
                        for post_period_length in post_period_length_list:
                            for decline_threshold in decline_threshold_list:
                                for decline_pct in decline_pct_list:
                                    for decline_type in ['threshold_mean','threshold_pct']:
                                        df_candidates = GetCandidates(departure_candidates, post_period_length, decline_type, decline_threshold, decline_pct, criteria_analysis_col)
                                        num_candidates = ContributorCount(df_candidates)
                                        df_candidates.to_parquet('issue/candidates.parquet')
                                        
                                        # add to a dataframe all these statistics about contributors
                                        print("total contributors: {}, potential (exceed thresholds) contributors: {}, \
                                              {} or more consecutive periods: {}, after removing duplicate exits: {}, \
                                              final candidate count, using {} post periods: {}".format(
                                                num_contributors, num_potential_contributors, consecutive_periods, num_consecutive_periods,
                                            num_departure_candidates, post_period_length, num_candidates))
    
                                        print(df_candidates[['actor_id','repo_name']].drop_duplicates().shape)
                                        print(df_candidates[['repo_name']].drop_duplicates().shape)
                                        print(np.sum(df_candidates[['actor_id','repo_name']].drop_duplicates()['repo_name'].value_counts()==1))
    
                